In [1]:
import requests
from bs4 import BeautifulSoup
import io
import time
import re
import pandas as pd

In [2]:
def sleeptime(hore,min,sec):
    return hore * 3600 + min * 60 + sec

In [3]:
url = "https://www.dcard.tw/f"
headers = {'User-Agent':'Mozilla/5.0(Windows NT 10.0;Win64;x64)AppleWebKit / 537.36 Chrome / 70.0.3538.102 Safari / 537.36'} 
resp = requests.get(url, headers = headers)
print(resp.status_code)

200


In [4]:
soup = BeautifulSoup(resp.text, 'html.parser')
dcard_title = soup.find_all('a', re.compile('sc-1v1d5rx-3 kPUUNB'))
dcard_title_s = []
for i, title in enumerate(dcard_title):
    print(str(i) + ". " + title.span.string)
    dcard_title_s.append(title.span.string)

In [5]:
import torch
from transformers import BertTokenizer
from IPython.display import clear_output
from transformers import BertForMaskedLM

PRETRAINED_MODEL_NAME = "bert-base-chinese"  # 指定繁簡中文 BERT-BASE 預訓練模型

# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

clear_output()
print("PyTorch 版本：", torch.__version__)

PyTorch 版本： 1.3.1


In [6]:
vocab = tokenizer.vocab
print(len(vocab))

21128


In [7]:
text = dcard_title_s[12]
print(text)
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)
tokens[5] = '[MASK]'
ids[5] = 103
print(tokens[:20], '...')
print(ids[:20], '...')


IndexError: list index out of range

In [71]:
tokens_tensor = torch.tensor([ids])
segments_tensors = torch.zeros_like(tokens_tensor)  # (1, seq_len)
maskedLM_model = BertForMaskedLM.from_pretrained(PRETRAINED_MODEL_NAME)
clear_output()

In [72]:
maskedLM_model.eval()
with torch.no_grad():
    outputs = maskedLM_model(tokens_tensor, segments_tensors)
    predictions = outputs[0]
del maskedLM_model

In [73]:
# 將 [MASK] 位置的機率分佈取 top k 最有可能的 tokens 出來
masked_index = 5
k = 5
probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k)
predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())

# 顯示 top k 可能的字。一般我們就是取 top 1 當作預測值
print("輸入 tokens ：", tokens[:10], '...')
print('-' * 50)
for i, (t, p) in enumerate(zip(predicted_tokens, probs), 1):
    tokens[masked_index] = t
    print("Top {} ({:2}%)：{}".format(i, int(p.item() * 100), tokens[:10]), '...')

輸入 tokens ： ['寵', '物', '真', '的', '有', '[MASK]', '情'] ...
--------------------------------------------------
Top 1 (21%)：['寵', '物', '真', '的', '有', '愛', '情'] ...
Top 2 ( 8%)：['寵', '物', '真', '的', '有', '純', '情'] ...
Top 3 ( 7%)：['寵', '物', '真', '的', '有', '色', '情'] ...
Top 4 ( 4%)：['寵', '物', '真', '的', '有', '戀', '情'] ...
Top 5 ( 4%)：['寵', '物', '真', '的', '有', '感', '情'] ...
